# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Country,Date,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mount Isa,AU,1622171399,35,22,-20.7333,139.5000,74.97,10.36
1,Mataura,NZ,1622171399,100,94,-46.1927,168.8643,49.48,4.16
2,Busselton,AU,1622171399,1,58,-33.6500,115.3333,73.04,14.81
3,Kattivākkam,IN,1622171400,33,61,13.2167,80.3167,89.08,13.20
4,Parrita,CR,1622171400,100,87,9.5201,-84.3225,82.17,2.21
...,...,...,...,...,...,...,...,...,...
597,Assiut,EG,1622171586,0,32,27.1810,31.1837,78.53,8.75
598,Veinticinco de Diciembre,MX,1622171586,74,11,25.2602,-104.0519,92.75,6.78
599,Shakawe,BW,1622171587,0,45,-18.3667,21.8500,48.20,3.56
600,Itoman,JP,1622171587,75,87,26.1247,127.6694,86.38,17.27


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]

In [4]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Country,Date,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed
205,Ţahţā,EG,1622171464,0,29,26.7693,31.5021,79.09,9.66
219,Yeppoon,AU,1622171469,0,27,-23.1333,150.7333,73.18,3.76
316,Puerto Peñasco,MX,1622171222,0,78,31.3167,-113.5333,76.91,8.05
322,Bereket,TM,1622171503,0,50,39.2446,55.5154,74.23,3.04
357,Saint-Pierre,RE,1622171513,0,76,-21.3393,55.4781,71.24,6.91
443,Joshīmath,IN,1622171539,0,30,30.5667,79.5667,73.78,2.80
444,Bāfq,IR,1622171539,0,23,31.6128,55.4107,79.63,2.21
456,São João da Barra,BR,1622171543,0,90,-21.6403,-41.0511,70.65,3.62
474,Yangi Marg`ilon,UZ,1622171549,0,33,40.4272,71.7189,78.22,0.00
597,Assiut,EG,1622171586,0,32,27.1810,31.1837,78.53,8.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
205,Ţahţā,EG,26.7693,31.5021,
219,Yeppoon,AU,-23.1333,150.7333,
316,Puerto Peñasco,MX,31.3167,-113.5333,
322,Bereket,TM,39.2446,55.5154,
357,Saint-Pierre,RE,-21.3393,55.4781,
443,Joshīmath,IN,30.5667,79.5667,
444,Bāfq,IR,31.6128,55.4107,
456,São João da Barra,BR,-21.6403,-41.0511,
474,Yangi Marg`ilon,UZ,40.4272,71.7189,
597,Assiut,EG,27.1810,31.1837,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 205: Ţahţā.
Closest hotel in Ţahţā is El Salam Hotel.
------------
Retrieving Results for Index 219: Yeppoon.
Closest hotel in Yeppoon is Salt Yeppoon.
------------
Retrieving Results for Index 316: Puerto Peñasco.
Closest hotel in Puerto Peñasco is Peñasco Del Sol Hotel and Conference Center.
------------
Retrieving Results for Index 322: Bereket.
Missing field/result... skipping.
------------
Retrieving Results for Index 357: Saint-Pierre.
Closest hotel in Saint-Pierre is Le Battant Des Lames.
------------
Retrieving Results for Index 443: Joshīmath.
Closest hotel in Joshīmath is The Tattva - A Boutique Resort.
------------
Retrieving Results for Index 444: Bāfq.
Missing field/result... skipping.
------------
Retrieving Results for Index 456: São João da Barra.
Closest hotel in São João da Barra is Pousada Kactus.
------------
Retrieving Results for Index 474: Yangi Marg`ilon.
Closest hotel in Yangi Marg`ilon is Diyor hotel.
------------
Retrieving Result

In [9]:
# Display the hotel dataframe
hotel_df


,City,Country,Lat,Lng,Hotel Name
205,Ţahţā,EG,26.7693,31.5021,El Salam Hotel
219,Yeppoon,AU,-23.1333,150.7333,Salt Yeppoon
316,Puerto Peñasco,MX,31.3167,-113.5333,Peñasco Del Sol Hotel and Conference Center
322,Bereket,TM,39.2446,55.5154,
357,Saint-Pierre,RE,-21.3393,55.4781,Le Battant Des Lames
443,Joshīmath,IN,30.5667,79.5667,The Tattva - A Boutique Resort
444,Bāfq,IR,31.6128,55.4107,
456,São João da Barra,BR,-21.6403,-41.0511,Pousada Kactus
474,Yangi Marg`ilon,UZ,40.4272,71.7189,Diyor hotel
597,Assiut,EG,27.1810,31.1837,Assiut hotels Armed Forces


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))